In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
user = pd.read_csv('../Data/whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # shape(31176,5834) 유저 x 상품 아이템
user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # shape (31177, 3250) 유저 x 영상 아이템
null_del_user_place = pd.read_csv('../Data/whyout_data/null_del_user_place.csv')
null_del_user_product = pd.read_csv('../Data/whyout_data/null_del_user_product.csv')
null_del_user_video = pd.read_csv('../Data/whyout_data/null_del_user_video.csv')
index_null_del_user_place = pd.read_csv('../Data/whyout_data/index_null_del_user_place.csv', index_col=0)
index_null_del_user_product = pd.read_csv('../Data/whyout_data/index_null_del_user_product.csv', index_col=0)
index_null_del_user_video = pd.read_csv('../Data/whyout_data/index_null_del_user_video.csv', index_col=0)
normal_index_null_del_user_place = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_place.csv', index_col=0)
normal_index_null_del_user_product = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_product.csv', index_col=0)
normal_index_null_del_user_video = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_video.csv', index_col=0)


In [ ]:
# # 나이 이상한 값 찾기
# find_user_nan = user.index[user['나이'].isna() | (user['나이'] >= 100) | (user['나이'] <= 0)].tolist()

# #print(find_user_nan)
# print('전체 user 수:',len(user))
# print('나이가 이상한 user 수:',len(find_user_nan))

In [ ]:
# # 나이 그룹 분포 확인
# df = user
# df = df.dropna(subset=['나이'])
# bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
# labels = ['유아', '10대', '20대', '30대', '40대', '50대', '60대', '70대', '80대']

# df['Age Group'] = pd.cut(df['나이'], bins=bins, labels=labels, right=False)
# age_group_distribution = df['Age Group'].value_counts().sort_index()

# # Display the result
# print(age_group_distribution)
# df = df[(df['나이'] > 0) & (df['나이'] < 100)]
# plt.figure(figsize=(16, 8))
# plt.hist(df['나이'], bins=30, color='blue', edgecolor='black')
# plt.title('Age Distribution')
# plt.xlabel('Age')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()
# # df.to_csv('age_group_distribution.csv', encoding='utf-8-sig')

In [5]:
# 사용자의 평가 유무 확인
def find_zero_indices(df):
    return df.index[df.eq(0).all(axis=1)].tolist()

place_zero_indices = find_zero_indices(user_place)
product_zero_indices = find_zero_indices(user_product)
video_zero_indices = find_zero_indices(user_video)

common_zero_indices = set(place_zero_indices) & set(product_zero_indices) & set(video_zero_indices)
#place_product_zero_indices = set(place_zero_indices) & set(product_zero_indices)
#place_video_zero_indices = set(place_zero_indices) & set(video_zero_indices)
#product_video_zero_indices = set(product_zero_indices) & set(video_zero_indices)

#print('장소를 평가하지 않은 사용자 수:',len(place_zero_indices))
#print('상품을 평가하지 않은 사용자 수:',len(product_zero_indices))
#print('영상을 평가하지 않은 사용자 수:',len(video_zero_indices))
print('모두 평가하지 않은 사용자 수:',len(common_zero_indices))
#print('장소 & 상품을 같이 평가하지 않은 사용자 수:',len(place_product_zero_indices))
#print('장소 & 영상을 같이 평가하지 않은 사용자 수:',len(place_video_zero_indices))
#print('상품 & 영상을 같이 평가하지 않은 사용자 수:',len(product_video_zero_indices))

모두 평가하지 않은 사용자 수: 5349


In [6]:
# 모두 평가하지 않은 사용자 행 삭제
# user_place.drop(common_zero_indices, inplace=True)
# user_product.drop(common_zero_indices, inplace=True)
# user_video.drop(common_zero_indices, inplace=True)

In [ ]:
# # idx missing value 찾기
# df = video.sort_values('idx')
# # 연속된 숫자 생성 (전체 범위의 시작과 끝을 기반으로)
# full_range = pd.Series(range(df['idx'].min(), df['idx'].max() + 1))
# # 누락된 숫자 찾기
# missing_numbers = full_range[~full_range.isin(df['idx'])]
# print(len(missing_numbers))

In [ ]:
# # NaN 값 찾기
# nan_indices = np.where(place['주변환경'].isna())[0]
# count = 0

# if len(nan_indices) > 0:
#     print(f'총 행 수: {len(place)}, NaN 행 수: {len(nan_indices)}')
#     print("해당 열에서 NaN 값이 발견된 위치는 다음과 같습니다:")
#     for idx in nan_indices:
#         print(f"인덱스: {idx}, 값: NaN")
# else:
#     print("해당 열에 NaN 값이 없습니다.")

In [20]:
# # 행, 열 이름 생성
# new_index = [str(i+1) for i in range(null_del_user_video.shape[0])]
# null_del_user_video.index = new_index

# new_columns = [str(i+1) for i in range(null_del_user_video.shape[1])]
# null_del_user_video.columns = new_columns

# # 변경된 데이터프레임 저장
# null_del_user_video.to_csv('index_null_del_user_video.csv', index=True, encoding='utf-8-sig') 

In [ ]:
# 유저가 장소에 대한 평가 유무
# 0의 개수
# zero_count = (user_place == 0).sum(axis=1)
# total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_place != 0).sum(axis=1)
# total_non_zero_count = non_zero_count.sum()

# 장소 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

# print(f"0의 개수: {zero_count}")
# print(f"0의 총 개수: {total_zero_count}")
#print(f"0이 아닌 숫자의 개수: {non_zero_count}")
# print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"장소에 대한 평가가 없는 유저 수: {count_zero}")
print(f"장소에 대한 평가가 있는 유저 수: {count_non_zero}")

In [ ]:
# 유저가 상품에 대한 평가 유무
# 0의 개수
# zero_count = (user_product == 0).sum(axis=1)
# total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_product != 0).sum(axis=1)
# total_non_zero_count = non_zero_count.sum()

# 상품 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

# print(f"0의 개수: {zero_count}")
# print(f"0의 총 개수: {total_zero_count}")
# print(f"0이 아닌 숫자의 개수: {non_zero_count}")
# print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"상품에 대한 평가가 없는 유저 수: {count_zero}")
print(f"상품에 대한 평가가 있는 유저 수: {count_non_zero}")

In [ ]:
# 유저가 영상에 대한 평가 유무
# 0의 개수
# zero_count = (user_video == 0).sum(axis=1)
# total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_video != 0).sum(axis=1)
# total_non_zero_count = non_zero_count.sum()

# 영상 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

# print(f"0의 개수: {zero_count}")
# print(f"0의 총 개수: {total_zero_count}")
#print(f"0이 아닌 숫자의 개수: {non_zero_count}")
# print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"영상에 대한 평가가 없는 유저 수: {count_zero}")
print(f"영상에 대한 평가가 있는 유저 수: {count_non_zero}")

In [23]:
# # 유저별 최대값을 구하여 그 값으로 나누어 정규화
# df = index_null_del_user_place
# normalized_df = df.div(df.max(axis=1), axis=0)
# normalized_df.fillna(0, inplace=True)
# normalized_df.to_csv('normal_index_null_del_user_place.csv')